### The New Person Problem

<img src="images/who_dis.png" width=600>

<img src="images/new_person.png" width=600>

### The Blueprint Solution

In [7]:
import pandas as pd

likes = pd.DataFrame([
    [0, 1, 1, 0, 0, 0],
    [0, 1, 1, 1, 0, 0],
    [1, 0, 0, 1, 0, 0],
    [0, 1, 1, 0, 0, 1],
    [0, 0, 0, 1, 1, 1]], 
    columns=['twix', 'mars', 'reeses', 'skittles', 'snickers', 'lindt']
)

likes

,twix,mars,reeses,skittles,snickers,lindt
0,0,1,1,0,0,0
1,0,1,1,1,0,0
2,1,0,0,1,0,0
3,0,1,1,0,0,1
4,0,0,0,1,1,1


In [9]:
from sklearn.metrics.pairwise import euclidean_distances

pd.DataFrame(euclidean_distances(likes))

,0,1,2,3,4
0,0.000000,1.000000,2.000000,1.000000,2.236068
1,1.000000,0.000000,1.732051,1.414214,2.000000
2,2.000000,1.732051,0.000000,2.236068,1.732051
3,1.000000,1.414214,2.236068,0.000000,2.000000
4,2.236068,2.000000,1.732051,2.000000,0.000000


In [13]:
new = pd.DataFrame([[0, 0, 1, 0, 0, 1]])

In [14]:
pd.DataFrame(euclidean_distances(likes, new))

,0
0,1.414214
1,1.732051
2,2.000000
3,1.000000
4,1.732051


### Using real data again...

In [15]:
df = pd.read_csv("data/candy.csv")
df = df[df['review'] >= 5]

Prep for CountVectorizer:

In [18]:
df = df.groupby(["user"])["item"].apply(lambda x: ",".join(x))
df = pd.DataFrame(df)
df.head()

,item
user,
aaron67,"Kit Kat Minis Crisp Wafers in Milk Chocolate,R..."
aaron68,"Brookside Dark Chocolate Pomegranate Flavor,Re..."
aaron73,Dove Chocolate Promises Silky Smooth Sea Salt ...
abarker,"Reese's Peanut Butter Bunny,Ghirardelli Gourme..."
abigail04,"Kit Kat Minis Crisp Wafers in Milk Chocolate,P..."


In [19]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(tokenizer=lambda x: x.split(","), max_features=250)
X = cv.fit_transform(df['item'])

In [52]:
### test

def comma_splitter(x):
    return x.split(',')

cv = CountVectorizer(tokenizer=tokenizer)
X = cv.fit_transform(df['item'])

with open("cv.pkl", "wb") as f:
    pickle.dump(cv, f)
    
with open("cv.pkl", "rb") as f:
    cv = pickle.load(f)
    
cv.transform(X)

AttributeError: lower not found

In [20]:
X

<2493x142 sparse matrix of type '<class 'numpy.int64'>'
	with 12977 stored elements in Compressed Sparse Row format>

In [21]:
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=5)
nn.fit(X)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [22]:
neighbors = nn.kneighbors(X, return_distance=False)
neighbors

array([[   0,  906, 1764,  853,  373],
       [   1,  399, 2104,  766,  853],
       [   2,  729,  321,  902, 2071],
       ...,
       [2490,  612,  652,  483,  863],
       [2491,  789,  305,  114, 2370],
       [2492,  150,  253,  730, 1231]])

In [23]:
neighbors[0]

array([   0,  906, 1764,  853,  373])

In [24]:
candy = []
for n in neighbors[0]:
    c = df.iloc[int(n)].values[0].split(",")
    candy.extend(c)
    
list(set(candy))

["Reese's Peanut Butter Cup Pumpkins Milk Chocolate",
 'Kit Kat Minis Crisp Wafers in Milk Chocolate',
 'Nestle Baby Ruth Bar',
 'Dove Chocolate Silky Smooth Dark Chocolate Singles Bar',
 "Reese's Peanut Butter Heart",
 'Snickers Peanut Butter Squared Bars',
 'Twizzlers Twists Strawberry']

### Wrapping it up and putting a bow on it

In [25]:
df = pd.read_csv("data/candy.csv")
df = df[df['review'] >= 5]
df = df.groupby(["user"])["item"].apply(lambda x: ",".join(x))
df = pd.DataFrame(df)
df.head()

,item
user,
aaron67,"Kit Kat Minis Crisp Wafers in Milk Chocolate,R..."
aaron68,"Brookside Dark Chocolate Pomegranate Flavor,Re..."
aaron73,Dove Chocolate Promises Silky Smooth Sea Salt ...
abarker,"Reese's Peanut Butter Bunny,Ghirardelli Gourme..."
abigail04,"Kit Kat Minis Crisp Wafers in Milk Chocolate,P..."


In [40]:
class NNRecommender:
    def __init__(self, n_neighbors=5, separator=","):
        self.separator = separator
        self.cv = CountVectorizer(tokenizer=lambda x: x.split(separator))
        self.nn = NearestNeighbors(n_neighbors=n_neighbors)

    def __repr__(self):
        return f'NNRecommender(n_neighbors={self.nn.n_neighbors}, separator="{self.separator}")'
        
    def fit(self, X):
        self.X = X
        X = self.cv.fit_transform(X)
        self.nn.fit(X)
        return self

    def predict(self, X):
        Xp = []
        for Xi in X:
            Xt = self.cv.transform([Xi])
            neighbors = self.nn.kneighbors(Xt, return_distance=False)
            repos = []
            for n in neighbors[0]:
                r = self.X.iloc[int(n)].split(self.separator)
                repos.extend(r)
            repos = list(set(repos))
            repos = [r for r in repos if r not in Xi.split(self.separator)]
            Xp.append(repos)
        return Xp

In [41]:
model = NNRecommender(n_neighbors=5)
model.fit(df["item"])

NNRecommender(n_neighbors=5, separator=",")

In [42]:
df.sample(1)['item'].values

array(["Hershey's Natural Unsweetened Cocoa,Nestle Toll House Ultimatestm Chocolate Chip Lovers Cookie Bar Dough,Bouquet of Fruits Valentine Chocolate Dipped Strawberries,Airheads Xtremes Sweetly Sour Candy Rainbow Berry,Mars Candy Fun Size Mix Peanut And Peanut Butter Lovers,Haribo Juicy Gold Bears Gummi Candy"],
      dtype=object)

In [43]:
sweet = ["Airheads Xtremes Sweetly Sour Candy Rainbow Berry,Life Savers Five Flavor Gummies,Twizzlers Pull-N-Peel Candy Cherry"]

In [44]:
peanut = ["Reese's Peanut Butter Cups Miniatures,M&Ms Peanut Chocolate Candy,Reese's Peanut Butter Big Cup"]

In [45]:
model.predict(sweet)

[['Nestle Butterfinger Bites',
  "Werther's Original Caramel Hard Candies",
  'Trolli Sour Brite Eggs Candy',
  'Starburst Tropical Fruit Chews Candy',
  "Hershey's Whoppers Malted Milk Balls"]]

In [46]:
model.predict(peanut)

[["Reese's Peanut Butter Egg",
  "Reese's Outrageous King Size Bar",
  'Snickers Chocolate Bar']]

In [47]:
import pickle

In [48]:
with open("model.pkl", "wb") as f:
    pickle.dump(model, f)

AttributeError: Can't pickle local object 'NNRecommender.__init__.<locals>.<lambda>'